In [ ]:
import pathlib
import sys
from gymnasium import spaces

sys.path.append(str(pathlib.Path().absolute().parent))

from src.replay_memory import FastReplayBuffer
from src.env import FourRoomEnvWithTagging


In [ ]:
BUF_SIZE = 30000
env = FourRoomEnvWithTagging(n_imposters=2, n_crew=4, n_jobs=5)
m = FastReplayBuffer(max_size=BUF_SIZE, state_size=env.flattened_state_size, trajectory_size=2, n_agents=env.n_agents, n_imposters=2)
m.populate(env, 10000)

In [ ]:
batch = m.sample(5)

In [ ]:
batch.imposters